# 管理数据结构

> 本主题介绍了Milvus中的模式。架构用于定义集合的属性和其中的字段。

## 字段模式

字段模式是字段的逻辑定义。这是在定义集合模式和管理集合之前需要定义的第一件事。

Milvus在一个集合中只支持一个主键字段。

### 字段架构属性

### 字段架构属性

| 性能                            | 描述                                 | 注意                                                         |
| :------------------------------ | :----------------------------------- | :----------------------------------------------------------- |
| `name`                          | 要创建的集合中的字段的名称           | 数据类型：String。 强制性                                    |
| `dtype`                         | 字段的数据类型                       | 强制性                                                       |
| `description`                   | 外地说明                             | 数据类型：String。 任择                                      |
| `is_primary`                    | 是否将字段设置为主键字段             | 数据类型：布尔（`true`或`false`）。 主键字段为必填项         |
| `auto_id`（主键字段必须）       | 切换以启用或禁用自动ID（主键）分配。 | `True`或`False`                                              |
| `max_length`（VARCHAR字段必须） | 允许插入的字符串的最大长度。         | 【1，65，535】                                               |
| `dim`                           | 向量的维数                           | 数据类型：∈[1，32768]。 对于密集向量场是必需的。对于稀疏向量场省略。 |
| `is_partition_key`              | 此字段是否为分区键字段。             | 数据类型：布尔（`true`或`false`）。                          |

## 创建字段架构

为了降低数据插入的复杂性，Milvus允许您在字段模式创建期间为每个标量字段指定默认值，不包括主键字段。这表示如果在插入数据时将字段留空，则将应用为此字段指定的默认值。

创建常规字段模式：

In [3]:
from pymilvus import FieldSchema, DataType

In [4]:
id_field = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, description="primary id")
age_field = FieldSchema(name="age", dtype=DataType.INT64, description="age")
embedding_field = FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=128, description="vector")

# 下文创建了一个字段，并将其用作分区字段
position_field = FieldSchema(name="position", dtype=DataType.VARCHAR, max_length=256, is_partition_key=True)

使用默认字段值创建字段架构

In [8]:
from pymilvus import FieldSchema

fields = [
  FieldSchema(name="id", dtype=DataType.INT64, is_primary=True),
  # configure default value `25` for field `age`
  FieldSchema(name="age", dtype=DataType.INT64, default_value=25, description="age"),
  FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=128, description="vector")
]

### 支持的数据类型

`DataType`定义字段包含的数据类型。不同的字段支持不同的数据类型。

- 主键字段支持：

  - INT64：numpy.int64
  - VARCHAR：VARCHAR

- 标量字段支持：

  - BOOL：布尔（`true`或`false`）
  - INT8：numpy.int8
  - INT16：numpy.int16
  - INT32：numpy.int32
  - INT64：numpy.int64
  - FLOAT：numpy.float32
  - Double：numpy. double
  - VARCHAR：VARCHAR
  - JSON：[JSON](https://milvus.io/docs/use-json-fields.md)
  - 数组：数组

  JSON是一种复合数据类型。JSON字段包含键-值对。每个键都是一个字符串，值可以是数字、字符串、布尔值、数组或列表。有关详细信息，请参阅[JSON：一种新的数据类型](https://milvus.io/docs/use-json-fields.md)。

- 矢量场支持：

  - BINARY_VECTOR：将二进制数据存储为0和1的序列，用于图像处理和信息检索中的紧凑特征表示。
  - FLOAT_VECTOR：存储32位浮点数，通常用于科学计算和机器学习，用于表示真实的数字。
  - FLOAT16_VECTOR：存储16位半精度浮点数，用于深度学习和GPU计算，以提高内存和带宽效率。
  - BFLOAT16_VECTOR：存储精度较低的16位浮点数，但与Float32具有相同的指数范围，在深度学习中很受欢迎，可减少内存和计算需求，而不会显著影响精度。
  - SPARSE_FLOAT_VECTOR：存储非零元素及其对应索引的列表，用于表示稀疏向量。有关详细信息，请参阅[稀疏向量](https://milvus.io/docs/sparse_vector.md)。

  Milvus支持集合中的多个向量场。有关详细信息，请参阅[多向量搜索](https://milvus.io/docs/multi-vector-search.md)。

## 集合模式

集合架构是集合的逻辑定义。通常，您需要在定义集合架构和管理集合之前定义字段架构。

### 集合架构属性

| 性能  | 描述                         | 注意                                                         |
| :---- | :--------------------------- | :----------------------------------------------------------- |
| `field` | 集合中要创建的字段           | 强制性                                                       |
| `description` | 收藏品的描述                 | 数据类型：String。 任择                                      |
| `partition_key_field` | 设计用作分区键的字段的名称。 | 数据类型：String。 任择                                      |
| `enable_dynamic_field` | 是否启用动态架构             | 数据类型：布尔（`true`或`false`）。 可选，默认为`False`。 有关动态架构的详细信息，请参阅动态架构和管理集合的用户指南。 |

### 创建集合架构

In [9]:
from pymilvus import FieldSchema, CollectionSchema
id_field = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, description="primary id")
age_field = FieldSchema(name="age", dtype=DataType.INT64, description="age")
embedding_field = FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=128, description="vector")

# Enable partition key on a field if you need to implement multi-tenancy based on the partition-key field
position_field = FieldSchema(name="position", dtype=DataType.VARCHAR, max_length=256, is_partition_key=True)

# Set enable_dynamic_field to True if you need to use dynamic fields. 
schema = CollectionSchema(
    fields=[id_field, age_field, embedding_field], 
    auto_id=False, 
    enable_dynamic_field=True, 
    description="desc of a collection"
)

使用指定的架构创建集合：

In [11]:
from pymilvus import connections, db

conn = connections.connect(host="127.0.0.1", port=19530)

In [12]:
from pymilvus import Collection


collection_name1 = "tutorial_1"
collection1 = Collection(name=collection_name1, schema=schema, using='default', shards_num=2)

* 您可以使用shards_num定义分片号。
* 您可以通过在using中指定别名来定义要在其上创建集合的Milvus服务器。
* 如果您需要实现基于分区键的多租户，可以通过在字段上设置is_partition_key到True来启用字段上的分区键功能。
* 如果需要启用动态字段，可以通过在集合模式中设置enable_dynamic_field到True来启用动态模式。

您还可以使用 `Collection.construct_from_dataframe`，它自动从DataFrame生成集合架构并创建集合。

In [16]:
import random
import pandas as pd

nb = 10
dim = 1024

df = pd.DataFrame({
    "id": [i for i in range(nb)],
    "age": [random.randint(20, 40) for i in range(nb)],
    "embedding": [[random.random() for _ in range(dim)] for _ in range(nb)],
    "position": "test_pos"
})

df.shape

(10, 4)

In [17]:
collection, ins_res = Collection.construct_from_dataframe(
    'my_collection',
    df,
    primary_field='id',
    auto_id=False
    )

In [18]:
collection

<Collection>:
-------------
<name>: my_collection
<description>: 
<schema>: {'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'age', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1024}}, {'name': 'position', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}], 'enable_dynamic_field': False}

In [19]:
ins_res

(insert count: 10, delete count: 0, upsert count: 0, timestamp: 449793736878063627, success count: 10, err count: 0)

------